In [ ]:
import part4
from copy_utilities import read_universal_
import utilities
import part2
import part1
import os 
import numpy as np

In [ ]:
#import all file paths
#common words are stop words for each language
STOP_words_ES_file_path = read_universal_('stopwords_ES.txt','ES')
STOP_words_RU_file_path = read_universal_('stopwords_RU.txt','RU')

#training path 
RU_train = read_universal_('train','RU')
ES_train = read_universal_('train','ES')

# test file path dev set
RU_test_dev = read_universal_('dev.in','RU')
ES_test_dev = read_universal_('dev.in','RU')

# write out file path
RU_test_dev_write = read_universal_('dev.p4.out','RU')
ES_test_write = read_universal_('dev.p4.out','ES')

In [ ]:
#read stopwords
#return nested lsit of stop words
ES_stopwords = part4.read_stopwords(STOP_words_ES_file_path)
RU_stopwords = part4.read_stopwords(STOP_words_RU_file_path)
print (RU_stopwords)

In [ ]:
#remove all the russian and spanish stopwords
def remove_stopwords(dataset,stop_words,symbols_list=[]):
    original_dataset = []
    file_dataset = open(dataset,'r',encoding='utf-8')
    training_set = file_dataset.readlines()
    for line in training_set:
        # if to include \n
        if len(line) == 1:
            original_dataset.append("\n")
        else:
            content_line = line.split() #split into the "text" and the "tag/label" using line.split()
            if content_line[0] in stop_words: #if the word belongs to the list of stopwords or list of symbols, assign the label O
                content_line[1] = "O" #assign label O
                original_dataset.append(content_line) #append to the ES dataset
            elif content_line[0] in symbols_list:
                continue
                #print(content_line[0])
        #Remove empty lists within list
    Edataset = [ele for ele in original_dataset]
    return Edataset

In [ ]:
RU_wo_stopwords = remove_stopwords(RU_train,RU_stopwords)
print(RU_wo_stopwords)

In [ ]:
es_train = utilities.read_data_transmission(ES_train)
tags = utilities.count_tags_transmission(es_train)
tag_words = utilities.count_tag_words(es_train)
transmission_counts = part2.count_transmissions(es_train)
#print(transmission_counts)
t_params = part2.estimate_transmission_parameters(transmission_counts, tags)
print(t_params)
#e_params = estimate_emission_parameters_with_unk(tags, tag_words)
print("=======================")

stopwordsRU = part4.read_stopwords(STOP_words_RU_file_path)
modif_RU = part4.remove_stopwords_es(RU_train,stopwordsRU)
tags_ru = utilities.count_tags_transmission(modif_RU)
print(modif_RU)
print(part4.get_symbols(modif_RU))


# Label Smoothing
# https://www.pyimagesearch.com/2019/12/30/label-smoothing-with-keras-tensorflow-and-deep-learning/
#may need to modify for PART IV, also different cases for start/stop possibly
#TODO modify this
def smooth_labels_transmission(transmission_parameters,alpha_sm=0.1):
    smoothed_dict = transmission_parameters
    #alpha_sm = 0.1 #hyperparameter for label smoothing, default 0.1
    for entry in smoothed_dict: #for each dictionary
        for label in smoothed_dict[entry]: #for each label in each dictionary
            if label != 'START':
                smoothed_dict[entry][label] *= (1 - alpha_sm)
                smoothed_dict[entry][label] += (alpha_sm/(len(smoothed_dict[entry])-1)) #add by the smoothing factor/number of labels without START
            # there is no transition to START at all
    return smoothed_dict #return smoothed dictionary

print('The smoothed dictionary is below \n')

transition_with_smooth = smooth_labels_transmission(t_params)

In [ ]:
# same thing do the
#emission dictionary takes the form of the word-> label
test_RU_dev= utilities.read_dev(RU_test_dev)
e_params = part1.estimate_emission_parameters_with_unk(tags,tag_words)

def smooth_labels_emission(emission_dict,alpha_sm=0.1):
    smoothed_dict = emission_dict
    for label, word_prob_dict in emission_dict.items():
        for word in word_prob_dict.keys():
            smoothed_dict[label][word] *= (1 - alpha_sm)
            smoothed_dict[label][word] += (alpha_sm/(len(word_prob_dict)))
    return smoothed_dict

emission_portion = smooth_labels_emission(e_params)

In [ ]:
#viterbi output
def run_viterbi(training_path, test_path, output_path,alpha_sm,stop_words_file_path):
    train = utilities.read_data_transmission(training_path)
    #read the stopwords
    stop_word = part4.read_stopwords(stop_words_file_path)
    train =  remove_stopwords(training_path,stop_word)
    train_words = utilities.get_training_set_words(train)
    test = utilities.read_dev(test_path)
    tags = utilities.count_tags_transmission(train)
    tag_words = utilities.count_tag_words(train)
    transmission_counts = part2.count_transmissions(train)
    t_params = part2.estimate_transmission_parameters(transmission_counts, tags)
    t_params = smooth_labels_transmission(t_params,alpha_sm=alpha_sm)
    e_params = part1.estimate_emission_parameters_with_unk(tags, tag_words)
    e_params = smooth_labels_emission(e_params,alpha_sm=alpha_sm)
    prediction = part2.viterbi_loop(test, t_params, e_params, train_words)
    utilities.output_prediction(prediction, test, output_path)

In [ ]:
# smoothing where alpha = 0.1

!python3 EvalScript/evalResult.py RU/dev.out RU/dev.p4.out

In [ ]:
a = 0.3
for i in np.arange(0,a,0.005):
    if os.path.exists(RU_test_dev_write):
        os.remove(RU_test_dev_write)
    run_viterbi(RU_train,RU_test_dev,RU_test_dev_write,i)
    stream = os.popen('python3 EvalScript/evalResult.py RU/dev.out RU/dev.p4.out')
    output = stream.read()
    print(f'When smoothing parameter = {i}')
    print(output)

In [ ]:
TEST_file_RU = read_universal_('test.in','test/RU-test')
print(TEST_file_RU)
run_viterbi(RU_train,TEST_file_RU,'RU/test.p4.out',0.01)